In [1]:
import numpy as np
import math
from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes, GCD
import random

In [2]:
def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [3]:
def encrypt_rsa(m, e, N):
    assert m < N
    c = pow(m, e, N)
    return c

# Prerequisites

1. RSA + its prerequisites

# Theory

Let:  
$p, q$ = secret primes  
$N = pq $ = modulus  
$\varphi(N) = (p-1)(q-1)$  

**Task** For another known encryption exponent $x$, find $m$ using a known $(e,d)$ pair:  
$c = m^x \ mod \ N$ => **Find $m$**

What do you know?
* $(e,d)$ = known pair 
* $c$ = the ciphertext
* $N$ = the modulus
* $x$ = the other encryption exponent

## Solution 1: Mathematical weakness

Observe: $ed \equiv 1 \ mod \varphi(N) => ed - 1 = k\cdot \varphi(N)$  

Let $y = x^{-1} \ mod \ k\varphi(N)$ => $xy \equiv 1 \ mod \ \varphi(N) => c^y \equiv m^{xy} \equiv m^1 \equiv m \ mod \ N$

## Solution 2: Factorize N from (e,d)

Algorithm + Explanation: https://crypto.stackexchange.com/questions/62482/algorithm-to-factorize-n-given-n-e-d/62487#62487

# Code

In [5]:
def sol1(c, e, d, x, N):
    kphi = e*d -1
    y = inverse(x, kphi)
    m = pow(c, y, N)
    return m

In [16]:
def factor_N(N, e, d):
    #for any a => a^{ed-1} mod N = 1; Let f = ed - 1
    f = e*d-1
    t = f 
    #write f = 2^s * t with t odd
    s = 0
    while not (t & 1):
        t = t>>1
        s+=1
    assert t & 1 == True
    assert f%N == pow(2, s, N)*t % N
    
    i = s
    from Crypto.Util.number import sieve_base
    for a in sieve_base:
        #exclude the trivial case
        if GCD(a, N)!= 1:
            gc = GCD(a, N)
            p = N/gc
            q = N/p
            return p, q
        
        b = pow(a, t, N) 
        #make sure b !=1 
        if b == 1:
            continue
        while i!=1:
            c = pow(b, 2, N)
            if c !=1:
                b = c
                i-=1
            else: 
                break
        if b==N-1:
            continue
        p = GCD(b-1,N)
        q = N//p
        #test p and q
        if p * q == N:
            break
            
    return p, q
    

In [7]:
p = getPrime(1024)
q = getPrime(1024)
N = p*q
phi = (p-1)*(q-1)
e = 65537
d = inverse(e, phi)

In [8]:
x = getPrime(200)

In [12]:
m = bytes_to_long(b'secret')
c = encrypt_rsa(m, x, N)

In [14]:
m_decr = sol1(c, e, d, x, N)
long_to_bytes(m_decr)

b'secret'

In [17]:
pf, qf = factor_N(N, e, d)

In [22]:
print((pf, qf) == (p, q))
d_decr = inverse(x, (pf-1)*(qf-1))
m_decr = pow(c, d_decr, N)
print(long_to_bytes(m_decr))


True
b'secret'
